## Введение в обработку естественного языка

Домашнее задание №12

Урок 12. Модель Transformer-2

*Формат именования файла домашней работы: FIO_NLP_HW_N.ipynb, где N - номер домашнего задания*

**

Задание  
Реализовать суммаризацию текста  
Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации  
Проверить насколько хорошо она суммаризирует  
2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

## Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
## 1. Проверить насколько хорошо она суммаризирует

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.8 MB/s eta 0:00:00


In [2]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [3]:
import json
import random
import pandas as pd
import re

In [4]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [5]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [6]:
!pip install rouge
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Метод Lead-3 (экстрактивная суммаризация)

In [7]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [8]:
import razdel

def calc_lead_n_score(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области.
BLEU:  0.19177311186434495
ROUGE:  {'rouge-1': {'r': 0.37762764047433917, 'p': 0.22208274285774904, 'f': 0.23804097238957525}, 'rouge-2': {'r': 0.15833772153385062, 'p': 0.09647636782929753, 'f': 0.10027796832321115}, 'rouge-l': {'r': 0.34937017731940756, 'p': 0.2022959168891477, 'f': 0.21799992093276083}}


## Предобученный трансформер (абстрактивная суммаризация)

In [9]:
import torch

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# эта моделька выдает такие метрики:
# BLEU:  0.11558216336979876
# ROUGE:  {'rouge-1': {'r': 0.08189563357258457, 'p': 0.20574736867306526, 'f': 0.1130750992990827}, 'rouge-2': {'r': 0.020742867977852363, 'p': 0.05315087145969499, 'f': 0.029394838192958163}, 'rouge-l': {'r': 0.07533478701173801, 'p': 0.18622355914925573, 'f': 0.10329732152130491}}

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [11]:
# from transformers import T5ForConditionalGeneration, T5Tokenizer
# model_name = 'cointegrated/rut5-base-absum'
# model = T5ForConditionalGeneration.from_pretrained(model_name)
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# # model.cuda();

# эта моделька выдает такие метрики:
# BLEU:  0.11163972768692136
# ROUGE:  {'rouge-1': {'r': 0.11875156585560502, 'p': 0.31950732600732595, 'f': 0.17030373945735267}, 'rouge-2': {'r': 0.045545355782226064, 'p': 0.1294754673431144, 'f': 0.06619812143697679}, 'rouge-l': {'r': 0.11637061347465263, 'p': 0.31117399267399265, 'f': 0.16660003575364896}}

In [12]:
model.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [13]:
df_test = pd.DataFrame(test_records)

In [14]:
df_test

,url,text,title,summary,date
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08
3,https://www.gazeta.ru/culture/2020/03/01/a_129...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,Народная артистка РСФСР Надежда Бабкина в инте...,2020-03-01 16:50:06
4,https://www.gazeta.ru/business/2020/02/06/1294...,Депутат Верховной рады от партии «Слуга народа...,«Поддерживают Россию»: почему Киев не платит п...,Украина не должна выплачивать пенсии жителям Д...,2020-02-06 12:41:24
...,...,...,...,...,...
5765,https://www.gazeta.ru/culture/2019/11/01/a_127...,Стриминговый сервис Netflix на итальянском фес...,Новая «Игра престолов»? Netflix показал трейле...,Стриминговый сервис Netflix опубликовал трейле...,2019-11-01 17:10:22
5766,https://www.gazeta.ru/culture/2019/11/05/a_127...,Голливудская актриса Анджелина Джоли и звезда ...,«Это было ужасно»: Джоли эвакуировали со съемо...,Актеры Анджелина Джоли и Ричард Мэдден были эв...,2019-11-05 20:09:26
5767,https://www.gazeta.ru/science/2019/11/22_a_128...,Университет штата Огайо удалил пресс-релиз об ...,«Это камни»: ученого затравили за змей на Марсе,Университет штата Огайо удалил пресс-релиз о н...,2019-11-22 15:02:51
5768,https://www.gazeta.ru/science/2020/01/27_a_129...,Китайским коронавирусом могут быть заражены ок...,«До 200 тысяч зараженных»: как распространяетс...,Китайским коронавирусом могут быть заражены ок...,2020-01-27 13:47:36


In [15]:
df_m = df_test[:10]

In [16]:
df_m['summary_pred'] = df_m.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)

<ipython-input-16-d263e440c7d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m['summary_pred'] = df_m.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)


In [18]:
# AutoModelForSeq2SeqLM
calc_scores(df_m['summary'].values, df_m['summary_pred'].values, metric="all")

Count: 10
Ref: Работающие пенсионеры дождались индексации. Это следует из уже одобренных в первом чтении поправок в Конституцию. Так что работающие пенсионеры смогут получить замороженную с 2016 года прибавку к пенсии. В России их насчитывается более девяти миллионов. Эксперты называют это восстановлением справедливости.
Hyp: В Госдуму внесены поправки в Конституцию, которые могут привести к индексации пенсий работающим пенсионерам. Русская служба Би-би-си собрала наиболее интересные новости о реформе пенсионного обеспечения россиян:
BLEU:  0.11558216336979876
ROUGE:  {'rouge-1': {'r': 0.08189563357258457, 'p': 0.20574736867306526, 'f': 0.1130750992990827}, 'rouge-2': {'r': 0.020742867977852363, 'p': 0.05315087145969499, 'f': 0.029394838192958163}, 'rouge-l': {'r': 0.07533478701173801, 'p': 0.18622355914925573, 'f': 0.10329732152130491}}


In [19]:
# T5ForConditionalGeneration
calc_scores(df_m['summary'].values, df_m['summary_pred'].values, metric="all")

Count: 10
Ref: Работающие пенсионеры дождались индексации. Это следует из уже одобренных в первом чтении поправок в Конституцию. Так что работающие пенсионеры смогут получить замороженную с 2016 года прибавку к пенсии. В России их насчитывается более девяти миллионов. Эксперты называют это восстановлением справедливости.
Hyp: В Госдуму внесены поправки в Конституцию, которые могут привести к индексации пенсий работающим пенсионерам. Русская служба Би-би-си собрала наиболее интересные новости о реформе пенсионного обеспечения россиян:
BLEU:  0.11558216336979876
ROUGE:  {'rouge-1': {'r': 0.08189563357258457, 'p': 0.20574736867306526, 'f': 0.1130750992990827}, 'rouge-2': {'r': 0.020742867977852363, 'p': 0.05315087145969499, 'f': 0.029394838192958163}, 'rouge-l': {'r': 0.07533478701173801, 'p': 0.18622355914925573, 'f': 0.10329732152130491}}


Метрика при абстрактивной суммаризации оказалась чуть лучше чем при экстрактивной. Хотя метрика получилась не очень хорошая.